### GET Surface Temperature Image Collection

Dataset Used: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2

### IMPORTS

In [18]:
import time
from typing import Any

import ee
import geemap
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

### AUTHENTICATE AND INITIALIZE

In [19]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

### VARIABLES

In [20]:
DATASET_NAME: str = "LANDSAT/LC08/C02/T1_L2"
START_DATE: str = "2023-01-01"
END_DATE: str = "2024-05-13"
BANDS: list[str] = ["SR_B4", "SR_B3", "SR_B2"]
SCALE: int = 30
# ZONES: range = range(2, 10)
TARGET_ZONE = 9
ZONES = [4, 9]

### FUNCTIONS

In [21]:
def mask_landsat_clouds(image):
  """Masks clouds in a landsat image using the QA_PIXEL band.
  Args:
      image (ee.Image): A landsat image.
  Returns:
      ee.Image: A cloud-masked landsat image.
  """
  qa = image.select('QA_PIXEL')

  # Bits 3 and 4 are clouds and Cloud Shadow, respectively.
  cloud_bit_mask = 1 << 3
  cloud_shadow_bit_mask = 1 << 4

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))
  )
  return image.updateMask(mask).select("S.*").copyProperties(image, ["system:time_start"])

  # return image.updateMask(mask).divide(10000)

In [22]:
# Applies scaling factors.
def apply_scale_factors(image):
    optical_bands = image.select("SR_B.").multiply(0.0000275).add(-0.2)
    thermal_bands = image.select("ST_B.*").multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

### GET IMAGES

In [23]:
shapefile_path = f"/vsizip/../shape_files/zone_{TARGET_ZONE}.zip/layers/POLYGON.shp"
gdf = gpd.read_file(shapefile_path)
roi = geemap.gdf_to_ee(gdf)

In [24]:
# roi.geometry()

In [25]:
LANDSAT = ee.ImageCollection(DATASET_NAME)

# With Clouds and Cloud Shadows
LANDSAT_ROI = LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)).filter(ee.Filter.bounds(roi.geometry())).map(apply_scale_factors)#.median()

# Create a cloud-free composite with default parameters.
COMPOSITE_ROI = LANDSAT_ROI.map(mask_landsat_clouds).median()

In [26]:
# Export with 30m resolution (refer to dataset)
image_ids = LANDSAT_ROI.aggregate_array("system:index").getInfo()
print("Total Images: ", len(image_ids))

Total Images:  84


### VISUALIZATION

In [33]:
visualization = {
    "bands": BANDS,
    "min": 0.0,
    "max": 0.3,
}

# Set opacity level between 0 (transparent) and 1 (opaque)
opacity_level = 0.6  
# Draw boundaries of the shapefile region loaded in 'table'
boundary = ee.FeatureCollection(roi)

Map = geemap.Map()
# m.set_center(9.188540, 45.464664, 9)
Map = geemap.Map(center=[45.52220212673159, 9.192582555499026], zoom=12)
# Map.set_center(9.143192990831077, 45.53639619976871, 10)
Map.add_layer(LANDSAT_ROI, visualization, "True Color (432)")
Map.add_layer(COMPOSITE_ROI, visualization, 'COMPOSITE_ROI')
Map.addLayer(boundary, {"color": "000000"}, "Region Boundary", True, opacity_level)
Map.addLayerControl()
Map

Map(center=[45.52220212673159, 9.192582555499026], controls=(WidgetControl(options=['position', 'transparent_b…

### DOWNLOAD IMAGES

In [17]:
for zone in tqdm(ZONES):
    shapefile_path = f"/vsizip/../shape_files/zone_{zone}.zip/layers/POLYGON.shp"
    gdf = gpd.read_file(shapefile_path)
    roi = geemap.gdf_to_ee(gdf)

    LANDSAT = ee.ImageCollection(DATASET_NAME)

    # With Clouds and Cloud Shadows
    LANDSAT_ROI = (
        LANDSAT.filter(ee.Filter.date(START_DATE, END_DATE)) \
        .filter(ee.Filter.bounds(roi.geometry())) \
        .map(apply_scale_factors)
    )  # .median()

    # Create a cloud-free composite with default parameters.
    COMPOSITE_ROI = LANDSAT_ROI.map(mask_landsat_clouds)

    # Median
    median = COMPOSITE_ROI.median()

    # EXPORTING THE MEDIAN IMAGE
    task_median = ee.batch.Export.image.toDrive(
        image=median.select("ST_B10"),  # Band 10 surface temperature
        description=f"Median_Image_Zone{zone}",
        fileNamePrefix=f"zone{zone}_median_stemp_{START_DATE}_{END_DATE}",
        folder="surface_temperature_roi_median_30m",
        scale=SCALE,
        region=roi.geometry().getInfo()["coordinates"],
        maxPixels=1e10
    )
    
    task_median.start()
    while task_median.active():
        print("Polling for task (id: {}).".format(task_median.id))
        time.sleep(5)  # Poll every 5 seconds

    
    print(f"Task for Zone {zone} median image download completed with state {task_median.status()['state']}: ")

    # EXPORTING ALL IMAGES WITH 30m RESOLUTION (refer to dataset)
    image_ids = COMPOSITE_ROI.aggregate_array("system:index").getInfo()
    print("#" * 16)
    print("Total Images: ", len(image_ids))
    print("#" * 16)

    for i, image_id in enumerate(tqdm(image_ids, desc="Exporting Images")):
        image = ee.Image(COMPOSITE_ROI.filter(ee.Filter.eq("system:index", image_id)).first())
        task = ee.batch.Export.image.toDrive(
        **{
            "image": image.select("ST_B10"), # Band 10 surface temperature.
            "description": "Image Export {}".format(i + 1),
            "fileNamePrefix": f"zone{zone}_{image.id().getInfo()}",
            "folder": "surface_temperature_roi_30m",
            "scale": SCALE,
            "region": roi.geometry().getInfo()["coordinates"], # Use ROI geometry
            # "region": image.geometry().bounds().getInfo()["coordinates"],
            "maxPixels": 1e10,
        })
        task.start()
        # Monitor the task
        while task.active():
            print("Polling for task (id: {}).".format(task.id))
            time.sleep(5)  # Poll every 5 seconds
        
        print(f"Task {i + 1} completed with state {task.status()['state']}: ")    

  0%|          | 0/2 [00:00<?, ?it/s]

Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).
Polling for task (id: Q5AVXTCL4MIAOB3MMNTNK6OA).


 50%|█████     | 1/2 [00:59<00:59, 59.00s/it]

Task for 4 median image download completed with state COMPLETED: 
Polling for task (id: 7PDO55WF23IBU2DM6KGERUE4).
Polling for task (id: 7PDO55WF23IBU2DM6KGERUE4).
Polling for task (id: 7PDO55WF23IBU2DM6KGERUE4).
Polling for task (id: 7PDO55WF23IBU2DM6KGERUE4).
Polling for task (id: 7PDO55WF23IBU2DM6KGERUE4).


100%|██████████| 2/2 [01:28<00:00, 44.04s/it]

Task for 9 median image download completed with state COMPLETED: 
